In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import StackingClassifier
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE 
standardscalar=StandardScaler()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [4]:
df_train=pd.read_csv('Train_Set_90621.csv')
df_test=pd.read_csv('Test_Set_90621.csv')

In [5]:
df_train['Documents Provided']=df_train['Documents Provided'].fillna('N')
df_train['Carry-forward Credit']=df_train['Carry-forward Credit'].fillna('N')
df_train['BankState']=df_train['BankState'].fillna('UNKNOWN')
df_train['Bank Type']=df_train['Bank Type'].fillna('GOVT')
df_train['Business Owner State']=df_train['Business Owner State'].fillna('UNKNOWN')
df_train['New Business']=df_train['New Business'].fillna('0.0')
df_train['Expected Company Income']=df_train['Expected Company Income'].fillna(df_train['Expected Company Income'].mean())

In [6]:
df_test['Documents Provided']=df_test['Documents Provided'].fillna('N')
df_test['Carry-forward Credit']=df_test['Carry-forward Credit'].fillna('N')
df_test['BankState']=df_test['BankState'].fillna('UNKNOWN')
df_test['Bank Type']=df_test['Bank Type'].fillna('GOVT')
df_test['Business Owner State']=df_test['Business Owner State'].fillna('UNKNOWN')
df_test['New Business']=df_test['New Business'].fillna('0.0')
df_test['Expected Company Income']=df_test['Expected Company Income'].fillna(df_test['Expected Company Income'].mean())

In [7]:
df_train['Expected Company Income']=df_train['Expected Company Income'].astype('int64')

In [8]:
df_train=df_train.drop(columns=['Application ID','Bank Masked','Name Masked','Approved_Timestamp'],axis=1)
df_test=df_test.drop(columns=['Application ID','Bank Masked','Name Masked','Approved_Timestamp'],axis=1)

In [9]:
df_cat_train=df_train.select_dtypes('object')
df_cat_test=df_test.select_dtypes('object')

In [10]:
df_cat_train=df_cat_train.drop(columns=['Business Owner State','BankState'],axis=0)
df_cat_test=df_cat_test.drop(columns=['Business Owner State','BankState'],axis=0)

In [11]:
df_cat_train_x1 = pd.get_dummies(df_cat_train, drop_first=True)
df_cat_test_x1 = pd.get_dummies(df_cat_test, drop_first=True)

In [13]:
df_train['Business Owner State']=le.fit_transform(df_train['Business Owner State'])
df_train['BankState']=le.fit_transform(df_train['BankState'])

df_test['Business Owner State']=le.fit_transform(df_test['Business Owner State'])
df_test['BankState']=le.fit_transform(df_test['BankState'])

In [14]:
XTrain=df_train.select_dtypes('int64')
XTest=df_test.select_dtypes('int64')

In [15]:
xtrain=pd.concat([XTrain,df_cat_train_x1],axis=1)
xtest=pd.concat([XTest,df_cat_test_x1],axis=1)

In [16]:
xi=xtrain.drop('Default_Status',axis=1)
xc=xtest

In [17]:
y=df_train['Default_Status']

In [18]:
x_train_scaled=standardscalar.fit_transform(xi)
x_test_scaled=standardscalar.fit_transform(xc)

In [19]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()

In [20]:
sm = SMOTE(random_state=123)
X_sm , y_sm = sm.fit_resample(x_train_scaled,y)

In [23]:
x_train , x_test , y_train ,y_test = train_test_split(X_sm,y_sm,test_size= 0.33)

In [30]:
logreg.fit(x_train,y_train)
#Predicting variable for the test dataset
y_pred_train = logreg.predict(x_test)
#y_pred_train = logreg.predict(x_test_scaled)

C:\Users\AS20188795\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
print(metrics.classification_report(y_test, y_pred_train))

              precision    recall  f1-score   support

           0       0.78      0.85      0.81     52655
           1       0.83      0.76      0.80     52866

    accuracy                           0.81    105521
   macro avg       0.81      0.81      0.81    105521
weighted avg       0.81      0.81      0.81    105521



In [24]:
import xgboost as xg
from xgboost import XGBClassifier

In [26]:
clf2 = xg.XGBClassifier().fit(x_train,y_train)
y_pred_xgb=clf2.predict(x_test_scaled)

[17:08:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\AS20188795\Anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [27]:
submission = pd.read_csv('Sample_Sub.csv')
submission['Default_Status']=y_pred_xgb
submission.to_csv('Sample_Sub.csv', index=False)